In [20]:
#Importamos las librerias necesarias
import pandas as pd
import pickle
import numpy as np
from scipy.signal import welch
import mne
from statistics import mean
import pyeeg as pe

In [26]:
#Abrimos los datos .dat, y convertimos a un DF
band = [.5,4,8,13,30,45]
start = 0
meta = []

with open('datos\s01.dat', 'rb') as file:
    subject = pickle.load(file, encoding='latin1')
    for i in range (0,40):
        data = subject["data"][i].transpose()
        df_dat = pd.DataFrame(data)
        df_dat =  df_dat.iloc[:,[1, 17, 7, 25, 12, 30, 14, 32]]
        df_dat.columns = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']
        labels = subject["labels"][i]
        start = 0
        while start + 256 < df_dat.shape[1]:
            meta_array = []
            meta_data = [] #meta vector for analysis
            for j in df_dat.columns():
                X = df_dat[j][start : start + 256] #Slice raw data over 2 sec, at interval of 0.125 sec
                Y = pe.bin_power(X, band, 128) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                meta_data = meta_data + list(Y[0])

            meta_array.append(np.array(meta_data))
            meta_array.append(labels)

            meta.append(np.array(meta_array))    
            start = start + 16
                
    meta = np.array(meta)
    np.save('datos', meta, allow_pickle=True, fix_imports=True)

In [25]:
#abrimos los datos .bdf y preprocesamos
#cargar los datos
filepath = 's01.bdf' #cambien a su filepath
raw = mne.io.read_raw_bdf(filepath)

#elegimos los canales que queramos
channels_names=['Fp1', 'Fp2', 'C3','C4', 'P7', 'P8','O1', 'O2']
raw.pick_channels(channels_names)
raw = raw.resample(128)
raw = raw.filter(4,45)
raw = raw.set_eeg_reference(ref_channels='average')
df_bdf = raw.to_data_frame(picks=list(range(raw.info['nchan'])))
df_bdf = df_bdf[['Fp1','Fp2','C3','C4', 'P7','P8','O1','O2']]

Extracting EDF parameters from c:\Users\migue\Documents\GitHub\NH\s01.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 213 samples (1.664 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
